# Relativistic GAN

ℹ️ Based on the work of [eriklindernoren](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/relativistic_gan/relativistic_gan.py).

In [ ]:
# Dataset path
dataroot = 'dataset/'
dataset_name = 'dog_photos' # e.g `dataroot/dataset_name`

# Number of workers for dataloader
workers = 0

# Batch size during training
batch_size = 60

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
# This setting is not actually usable as you need to manually adjust the
#   architecture of the models in case you wish to use a different image size
image_size =  256
img_size = image_size
hr_height = image_size
hr_width = image_size

# Number of channels in the training images. For color images this is 3
channels = 3

# Size of z latent vector (i.e. size of generator input)
latent_dim = 100

# Number of training epochs
n_epochs = 9000

# start with epoch
# time to time during the training the model state will be saved into `models` folder
# in case you need to break the execution adjust this value with the number of the last epochs
#   to continue where you left of last time.
start_with_epoch = 0

# Learning rate for optimizers
lr = 0.0002

# Beta hyperparams for Adam optimizers
b1 = 0.5
b2 = 0.999

# processing settings
save_state_each = 1 # save models state every / epochs
save_image_each = 2 # save intermediate result every / epochs
check_alive_every = 3 # display loss information every / epochs


In [ ]:
import argparse
import os
import numpy as np
import math
import glob

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.utils.data import Dataset
from PIL import Image

from torchvision.models import vgg19

import gc 

In [ ]:
# create folders
os.makedirs("images", exist_ok=True)
os.makedirs("model", exist_ok=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
hr_shape = (hr_height, hr_width)

In [ ]:
def checkpoint(name, data, epoch_num):
    torch.save(data, '%s/%s_epoch_%s.pth' % ('./model', name, epoch_num))    

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1))

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [ ]:
dataset = datasets.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
#[transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# save some resources
del dataset

In [ ]:
# Loss function
adversarial_loss = torch.nn.BCEWithLogitsLoss().to(device)

# Initialize generator and discriminator
generator = Generator().to(device)
discriminator = Discriminator().to(device)

if start_with_epoch != 0:
    print("load previous state")
    generator.load_state_dict(torch.load("./model/relativ_gan_gen_epoch_%d.pth" % start_with_epoch, map_location=device))
    discriminator.load_state_dict(torch.load("./model/relativ_gan_disc_epoch_%d.pth" % start_with_epoch, map_location=device))

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor



In [ ]:
# ----------
#  Training
# ----------

print('Training')
for epoch in range(start_with_epoch, n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        real_pred = discriminator(real_imgs).detach()
        fake_pred = discriminator(gen_imgs)
        
        g_loss = adversarial_loss(fake_pred - real_pred.mean(0, keepdim=True), valid)
        # for normal
        # g_loss = adversarial_loss(fake_pred - real_pred, valid)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Predict validity
        real_pred = discriminator(real_imgs)
        fake_pred = discriminator(gen_imgs.detach())
        
        real_loss = adversarial_loss(real_pred - fake_pred.mean(0, keepdim=True), valid)
        fake_loss = adversarial_loss(fake_pred - real_pred.mean(0, keepdim=True), fake)
        
        # for normal
        # real_loss = adversarial_loss(real_pred - fake_pred, valid)
        # fake_loss = adversarial_loss(fake_pred - real_pred, fake)
        
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # --------------
        #  Log Progress
        # --------------

        batches_done = epoch * len(dataloader) + i
        if batches_done % save_image_each == 0:
            print('save image')
            save_image(gen_imgs.data[:25], "images/relativ_gan_%d.png" % batches_done, nrow=5, normalize=True)
            
        if batches_done % check_alive_every == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            
        if batches_done % save_state_each == 0:
            print('save checkpoint')
            checkpoint('relativ_gan_gen', generator.state_dict(), epoch)
            checkpoint('relativ_gan_disc', discriminator.state_dict(), epoch)
            
    gc.collect()
    torch.cuda.empty_cache()
        
checkpoint('relativ_gan_gen', generator.state_dict(), 'final')
checkpoint('relativ_gan_disc', discriminator.state_dict(), 'final')      

print('finish')
save_image(gen_imgs.data[:25], "images/relativ_gan_final.png", nrow=5, normalize=True)
            

In [ ]:
# just in case we need to interrupt and save the results manually from the current memory
#checkpoint('relativ_gan_gen', generator.state_dict(), EPOCH_NUMBER_HERE)
#checkpoint('relativ_gan_disc', discriminator.state_dict(), EPOCH_NUMBER_HERE)
#save_image(gen_imgs.data[:25], "images/relativ_gan_epoch_%d.png" % EPOCH_NUMBER_HERE, nrow=5, normalize=True)